In [9]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def sigmoid(x):
    return 1/(1+np.exp(-x)) 

def sigmoid_grad(x):
    a = sigmoid(x)
    return a*(1-a)

def tanh(x):
    return np.tanh(x)

def tanh_grad(x):
    a = tanh(x)
    return 1 - a*a

def relu(x):
    x[x<0] = 0
    return x

def relu_grad(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [11]:
activations = {
    "sigmoid": [sigmoid,sigmoid_grad],
    "tanh": [tanh,tanh_grad],
    "relu": [relu,relu_grad]
}

In [12]:
def cross_entropy_loss(real,pred):
    ret = -np.sum(np.multiply(read,np.log(pred)) , axis = 1)
    return ret

def cross_entropy_loss_grad(real,pred):
    return (pred-real)/((pred+1e-8)*(1-pred))

def mse_loss(real,pred):
    ret = np.mean((pred-real)**2)/2
    return ret
    
def mse_loss_grad(real,pred):
    return pred-real

In [13]:
losses = {
    "cross_entropy":(cross_entropy_loss,cross_entropy_loss_grad),
    "mse": (mse_loss,mse_loss_grad)
}

In [14]:
class layer:
    
    def __init__(self,neurons,inputs,activation = None):
        self.W = np.random.randn(neurons,inputs)
        self.b = np.random.randn(neurons,)
        self.neurons = neurons
        self.inputs = inputs
        if activation is None:
            self.activation = activations["relu"]
        else:
            self.activation = activations[activation]
            
    def run_layer(self,A):
        a = self.W @ A +self.b      #a is not dot product
        self.inp = A
        return self.activation[0](a)
    
    def back_pass(self,dl_da,lr=0.01):
        h = self.W @ A + self.b
        a = self.activation[0](h)
        dl_h = np.transpose(self.W) @ dl_da
        dl_da_curr = np.multiply(dl_h, self.activation[1](a))
        dl_w = dl_da_curr @ np.transpose(self.inp)
        dl_b = dl_a
        
        self.W -= lr * dl_w
        self.b -= lr* dl_b
        
        return dl_da_curr
        

In [15]:
class models:
    
    def __init__(self,inputs,loss=None,optimizer=None):
        self.inputs = inputs
        self.layers = []
        if loss is None:
            self.loss = losses["cross_entropy"]
        else:
            self.loss = losses[loss]
        self.optimizer = optimizer
        
        
    def add_layer(self,neurons,activation=None):
        if len(self.layers) == 0:
            inps = self.inputs
        else:
            inps = self.layers[-1].neurons
        new_layer = layer(neurons,inps,activation)
        self.layers.append(new_layer)
        
        
    def forward_pass(self,inputs):
        res = inputs
        for lay in self.layers:
            res = lay.run_layer(res)
            
        return res
    
    def print(self):
        for lay in self.layers:
            print("Layer : ")
            print("Input: {}".format(lay.inputs))
            print("Neurons: {}".format(lay.neurons))
            
    def backward_pass(self,dl_a):
        for lay in reverse(self.layers) :
            dl_da = lay.back_pass(self,dl_da)
    
    def predict(self,inputs):
        return self.forward_pass(inputs)
    
    
    def train(self,inputs,outputs,epochs):
        loss_tracker = []
        for i in range(epochs):
            pred = self.predict(inputs)
            loss = self.loss[0](outputs,pred)
            loss_tracker.append(loss)
            da = self.loss[1](outputs,pred)
            self.backward_pass(da)
            

In [19]:
from tensorflow.keras.datasets import fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

ModuleNotFoundError: No module named 'tensorflow'

In [17]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

_, indicesList = np.unique(y_train, return_index=True)

plt.figure(figsize=(10,10))
for i in range(10):
    plt.xticks([])
    plt.yticks([])
    plt.subplot(5,5,i+1)
#     plt.grid(False)
    plt.imshow(x_train[indicesList[i]] , cmap=plt.cm.binary)
    plt.xlabel(class_names[i])
plt.show()

NameError: name 'y_train' is not defined

In [164]:
def transx(x):
    l,b,h = x.shape
    xt = x.reshape((l,b*h))
    return xt

In [165]:
xt = transx(x_train)

In [158]:
def one_hot_encoding(a):
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b

yt = one_hot_encoding(y_train)
print(yt.shape)

(60000, 10)


In [172]:
model = models(784)
model.add_layer(64)
model.add_layer(32)
model.add_layer(len(class_names),activation="sigmoid")
model.print()

Layer : 
Input: 784
Neurons: 64
Layer : 
Input: 64
Neurons: 32
Layer : 
Input: 32
Neurons: 10


In [173]:
model.train(xt,yt,100)

<ipython-input-41-5712a88165bc>:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
<ipython-input-119-833cb4f86a09>:2: RuntimeWarning: divide by zero encountered in log
  ret = -(real*np.log(pred+1e-8) + (1-real)*np.log(1-pred))
<ipython-input-119-833cb4f86a09>:2: RuntimeWarning: invalid value encountered in multiply
  ret = -(real*np.log(pred+1e-8) + (1-real)*np.log(1-pred))
<ipython-input-119-833cb4f86a09>:6: RuntimeWarning: divide by zero encountered in true_divide
  return (pred-real)/((pred+1e-8)*(1-pred))
<ipython-input-119-833cb4f86a09>:6: RuntimeWarning: invalid value encountered in true_divide
  return (pred-real)/((pred+1e-8)*(1-pred))
<ipython-input-121-2f1fb0f2c122>:21: RuntimeWarning: invalid value encountered in multiply
  da = np.multiply(self.activation[1](a),dnext)
<ipython-input-41-5712a88165bc>:16: RuntimeWarning: invalid value encountered in less
  x[x<0] = 0
<ipython-input-41-5712a88165bc>:20: RuntimeWarning: invalid value encountered in less_

In [174]:
xte = transx(x_test)
pred = np.argmax(model.predict(xte))

<ipython-input-41-5712a88165bc>:16: RuntimeWarning: invalid value encountered in less
  x[x<0] = 0


In [175]:
print(np.sum(pred==y_test)/y_test.shape[0])

0.1
